![cover](./img/slide1.png)

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp

import plotly.offline as po
import plotly.graph_objs as go


import ipywidgets as widgets
from ipywidgets import interact

from ipywidgets.embed import embed_minimal_html


po.init_notebook_mode(connected=True)

In [ ]:
def call_put_vol(atm_vol,fly25_vol,rr25_vol):
    put_25 = (atm_vol+fly25_vol-(0.5*rr25_vol))/100
    atm = (atm_vol)/100
    call_25 = (atm_vol+fly25_vol+(0.5*rr25_vol))/100
       
    return [25,50,75],[call_25,atm,put_25]
    
def interpolate(X,Y):
    return sp.interpolate.interp1d(X,Y, kind='quadratic')

### <center>Compute call / put volatility</center>
$$ \begin{cases} \sigma_{nP} = \sigma_{ATM} + \sigma_{nFLY} - \frac{1}{2}\sigma_{nRR}\\
\sigma_{nC} = \sigma_{ATM} + \sigma_{nFLY} + \frac{1}{2}\sigma_{nRR}\\
\sigma_{ATM} = \sigma_{ATM}
\end{cases}$$

In [ ]:
layout = widgets.Layout(width='98%', height='50px')
style = {'description_width': 'initial'}

X,Y = call_put_vol(7.25,0.25,0.15)
x1 = np.linspace(25,75)
interp = interpolate(X,Y)
y1 = interp(x1)

#Creates widget containing a figure
fig1 = go.FigureWidget()
#Adds three different scatter plots
fig1.add_scatter(x= X, y = Y, name='Quoted volatilities',mode = 'markers',marker=dict(size=8,color='turquoise'))
fig1.add_scatter(x= x1, y = y1, name='Quoted volatilities', line=dict(color='turquoise', dash='dot'),showlegend=False)

fig1.add_scatter(x= X, y = Y, name='Mod. volatilities',mode = 'markers',marker=dict(size=8,color='steelblue'))
fig1.add_scatter(x= x1, y = y1, name='Mod. volatilities', line=dict(color='steelblue'),showlegend=False)


#Slider and Value box to move drift term
atmSlide = widgets.FloatSlider(value = 7.25, min=0, max=10, step=0.05, description='ATM: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
atmVal = widgets.FloatText(value = 0., step= 0.01,disabled = False)
atm = widgets.Box([atmSlide, atmVal])
atmLink = widgets.jslink((atmSlide,'value'),(atmVal, 'value'))

#Slider and Value box to move diffusion term
flySlide = widgets.FloatSlider(value = 0.25, min=0, max=1, step=0.05, description='25D FLY: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
flyVal = widgets.FloatText(value = 0.25, step= 0.05,disabled = False)
fly = widgets.Box([flySlide,flyVal])
flyLink = widgets.jslink((flySlide,'value'), (flyVal, 'value'))

#Slider and Value box to move diffusion term
rrSlide = widgets.FloatSlider(value = 0.15, min=0, max=1, step=0.05, description='25D RR: ', disabled=False,
    continuous_update=False, orientation='horizontal', readout=False, readout_format='.2f', layout=layout, style=style)
rrVal = widgets.FloatText(value = 0.15, step= 0.051,disabled = False)
rr = widgets.Box([rrSlide,rrVal])
rrLink = widgets.jslink((rrSlide,'value'), (rrVal, 'value'))

#Button to restore default values
reset = widgets.Button(description='Restore values')

button_box= widgets.HBox([reset])
box= widgets.VBox([atm,fly,rr,button_box])

def on_reset_click(change):
    flySlide.value=0.25
    rrSlide.value=0.15
    atmSlide.value=7.25
    return

reset.on_click(on_reset_click)

def f(atmVal,flyVal,rrVal):
            
    with fig1.batch_update():
        X1,Y1 = call_put_vol(atmVal,flyVal,rrVal)
        x2 = np.linspace(25,75)
        interp = interpolate(X1,Y1)
        fig1.data[2].y = Y1
        fig1.data[3].y = interp(x2)

out = widgets.interactive_output(f, { 'atmVal': atmVal, 'flyVal': flyVal,'rrVal': rrVal})

#Sets figure title and axis 
fig1.layout.title={'text':'FX Call & Put volatility','font':{'size': 25},'x':.5,'xanchor':'center'}
fig1.layout.yaxis={'fixedrange': False,'range':[.07,.08], 'title': 'Volatility','titlefont':{'size': 20},'tickformat':'.3p', 'hoverformat':',.5p'}
fig1.layout.xaxis={'title': 'Delta', 'tick0':0, 'titlefont':{'size': 20}}
fig1.layout.showlegend = True
fig1.update_layout(legend={'orientation':'h'})
fig1.layout.height = 650

In [ ]:
display(box, out, fig1)